## Codigo de practica

In [ ]:
import requests
from bs4 import BeautifulSoup

#third party advice

# 1. Obtener la URL
url = "https://books.toscrape.com/"
respuesta_obtenida = requests.get(url)
html_obtenido = respuesta_obtenida.text  # HTML como string

# 2. Parsear ese HTML con BeautifulSoup
soup = BeautifulSoup(html_obtenido, 'html.parser')  

#categorias

# # --- Método 1: usando ul_menu y for ---
# categorias = soup.find('ul', class_='nav nav-list')
# items = categorias.find_all('li')

# diccionario_links = {}

# for li in items:
#     enlace = li.find('a')
#     texto = enlace.text.strip()
#     link = enlace['href']
#     diccionario_links[texto] = link
    
# print('Método 1:', diccionario_links)

# # --- Método 2: usando comprensión de diccionario y select ---
# diccionario_links2 = {a.text.strip(): a['href'] for a in soup.select('ul.nav.nav-list li a')}
# print('Método 2:', diccionario_links2)

# --- Método 3: usando un bucle for y select, pero en menos líneas ---

diccionario_links3 = {}
for a in soup.select('ul.nav.nav-list li a'):
    diccionario_links3[a.text.strip()] = url + a['href']
print(diccionario_links3)


for link in diccionario_links3.keys():
    link = requests.get(url)
    link_text = link.text
    if link_text.find('ul', class_ = 'pager') == True:
        for page in link



SyntaxError: expected ':' (111805284.py, line 47)

In [ ]:
import requests
from bs4 import BeautifulSoup
libros = None


base_url = "https://books.toscrape.com/catalogue/page-{}.html"


#funcion para recorrer todas las paginas 

for page in range(1, 2):  # Hay 50 páginas en total
    url = base_url.format(page)
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')

    # Aquí haces scraping de los libros de esa página
    libros = soup.find_all('article', class_='product_pod')
    print(f"📄 Página {page} -> {len(libros)} libros encontrados")

# recorrer_paginas()


#   

for libro in libros:
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()

    # Enlace al detalle del libro (relativo -> convertir a absoluto)
    enlace_relativo = libro.h3.a['href']
    enlace_absoluto = "https://books.toscrape.com/catalogue/" + enlace_relativo

    print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 🔗 {enlace_absoluto}")
    
    
    
detalle = requests.get(enlace_absoluto)
soup_detalle = BeautifulSoup(detalle.text, 'html.parser')

# Categoría: está en un breadcrumb (<ul class="breadcrumb">)
categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text

print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 📂 {categoria}")

    
    





📄 Página 1 -> 20 libros encontrados
📚 A Light in the Attic | 💵 Â£51.77 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
📚 Tipping the Velvet | 💵 Â£53.74 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
📚 Soumission | 💵 Â£50.10 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/soumission_998/index.html
📚 Sharp Objects | 💵 Â£47.82 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sharp-objects_997/index.html
📚 Sapiens: A Brief History of Humankind | 💵 Â£54.23 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
📚 The Requiem Red | 💵 Â£22.65 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-requiem-red_995/index.html
📚 The Dirty Little Secrets of Getting Your Dream Job | 💵 Â£33.34 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
📚 The Coming Woman: A Novel Based on the Li

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor

def get_with_retries(url, headers=None, max_retries=3, timeout=5):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.status_code == 200:
                return response
        except requests.exceptions.RequestException as e:
            print(f'Intento {attempt+1} fallido para {url}: {e}')
            time.sleep(2)
    print(f'No se pudo acceder a {url} después de {max_retries} intentos.')
    return None

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
base_url = "https://books.toscrape.com/"

def get_google_books_authors(titulo):
    titulo_api = titulo.replace(" ", "%")
    api_url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{titulo_api}"
    try:
        resp_api = requests.get(api_url, timeout=10)
        data_api = resp_api.json()
        if "items" in data_api and len(data_api["items"]) > 0:
            return data_api["items"][0]["volumeInfo"].get("authors", [])
    except:
        pass
    return []

def process_book(libro, nombre_cat):
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()
    rating = libro.p['class'][1] if len(libro.p['class']) > 1 else 'None'
    
    enlace_rel = libro.h3.a['href']
    if enlace_rel.startswith('../'):
        enlace_rel = enlace_rel.replace('../', '')
    enlace_abs = base_url + 'catalogue/' + enlace_rel
    
    # Obtener detalles de la página del libro
    detalle = get_with_retries(enlace_abs, headers=headers)
    if detalle is None:
        return None
    
    soup_detalle = BeautifulSoup(detalle.text, 'html.parser')
    
    try:
        categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text.strip()
    except:
        categoria = nombre_cat
    
    try:
        stock = soup_detalle.find('p', class_='instock availability').text.strip()
    except:
        stock = ''
    
    # Obtener autores de Google Books (en paralelo)
    autores = get_google_books_authors(titulo)
    
    return {
        'Título': titulo,
        'Autores': autores,
        'Precio': precio,
        'Disponibilidad': disponibilidad,
        'Rating': rating,
        'Categoría': categoria,
        'Stock': stock,
        'URL': enlace_abs
    }

def main():
    # Obtener categorías
    respuesta = get_with_retries(base_url, headers=headers)
    if respuesta is None:
        raise Exception('No se pudo obtener la página principal')
    
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    categorias = {
        a.text.strip(): base_url + a['href'] 
        for a in soup.select('ul.nav.nav-list li a') 
        if a['href'] != 'catalogue/category/books_1/index.html'
    }
    
    print(f"Categorías encontradas: {len(categorias)}")
    
    libros_data = []
    contador = 0
    
    for nombre_cat, url_cat in categorias.items():
        page = 1
        while True:
            # URL corregida para paginación
            if 'index.html' in url_cat:
                base_url_cat = url_cat.replace('index.html', '')
            else:
                base_url_cat = url_cat[:-1] if url_cat.endswith('/') else url_cat
            
            url_pagina = f"{base_url_cat}/page-{page}.html" if page > 1 else url_cat
            
            resp_cat = get_with_retries(url_pagina, headers=headers)
            if resp_cat is None:
                break
                
            soup_cat = BeautifulSoup(resp_cat.text, 'html.parser')
            libros = soup_cat.find_all('article', class_='product_pod')
            
            if not libros:
                break
            
            with ThreadPoolExecutor(max_workers=3) as executor:
                futures = [executor.submit(process_book, libro, nombre_cat) for libro in libros]
                
                for future in futures:
                    libro_info = future.result()
                    if libro_info:  # Ahora solo se filtra si process_book devuelve None explícitamente
                        libros_data.append(libro_info)
                        contador += 1
                        print(f"📖 Libro {contador}: {libro_info}")
            
            # Verificar si hay más páginas
            next_page = soup_cat.find('li', class_='next')
            if not next_page:
                break
                
            page += 1
            time.sleep(0.5)  # Pequeña pausa entre páginas
    
    return libros_data

if __name__ == "__main__":
    start_time = time.time()
    data = main()
    end_time = time.time()
    print(f"\nTiempo total de ejecución: {end_time - start_time:.2f} segundos")
    print(f"Total de libros obtenidos: {len(data)}")

# # Guardamos todos los libros recolectados en un archivo CSV
# with open('libros_books_to_scrape.csv', 'w', newline='', encoding='utf-8') as f:
#     writer = csv.DictWriter(f, fieldnames=libros_data[0].keys())
#     writer.writeheader()
#     writer.writerows(libros_data)

# print(f"Total de libros guardados: {len(libros_data)}")





Categorías encontradas: 50
📖 Libro 1: {'Título': "It's Only the Himalayas", 'Autores': ['S. Bedford'], 'Precio': 'Â£45.17', 'Disponibilidad': 'In stock', 'Rating': 'Two', 'Categoría': 'Travel', 'Stock': 'In stock (19 available)', 'URL': 'https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html'}
📖 Libro 2: {'Título': 'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Autores': ['Rick Antonson'], 'Precio': 'Â£49.43', 'Disponibilidad': 'In stock', 'Rating': 'Four', 'Categoría': 'Travel', 'Stock': 'In stock (15 available)', 'URL': 'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html'}
📖 Libro 3: {'Título': 'See America: A Celebration of Our National Parks & Treasured Sites', 'Autores': ['Creative Action Network'], 'Precio': 'Â£48.87', 'Disponibilidad': 'In stock', 'Rating': 'Three', 'Categoría': 'Travel', 'Stock': 'In stock (14 available)', 'URL': 'https://books.toscrape.com/catalogu

NameError: name 'libros_data' is not defined

## Codigo funcional de scraping


## DIAGRAMAS UML DE LA BASE DE DATOS 

┌─────────────────────────────┐
│         Categoría           │
├─────────────────────────────┤
│ + id : INT (PK)             │
│ + nombre : VARCHAR(100)     │
│ + url : TEXT                │
└───────────────┬─────────────┘
                │ (1:N)
                │
┌───────────────▼─────────────┐
│           Libro             │
├─────────────────────────────┤
│ + id : INT (PK)             │
│ + titulo : VARCHAR(255)     │
│ + precio : DECIMAL(10,2)    │
│ + disponibilidad : VARCHAR  │
│ + rating : INT              │
│ + stock : INT               │
│ + url : TEXT                │
│ + descripcion : TEXT        │      │
│ + categoria_id : INT (FK)   │──┐
└───────────────┬─────────────┘  │
                │ (N:M)           │
                │                 │
┌───────────────▼─────────────┐   │
│          Libro_Autor        │   │
├─────────────────────────────┤   │
│ + libro_id : INT (FK)       │◄──┘
│ + autor_id : INT (FK)       │
└───────────────┬─────────────┘
                │ (1:N)
┌───────────────▼─────────────┐
│           Autor             │
├─────────────────────────────┤
│ + id : INT (PK)             │
│ + nombre : VARCHAR(150)     │
└─────────────────────────────┘

## N:M que se puede representar a futuro en el proyecto

[Categoría] 1 ────────────< N [Libro] >────────────< N ──────────── 1 [Autor]
                            │
                            │ (Tabla intermedia)
                            ▼
                      [Libro_Autor]






In [5]:
import pandas as pd


# Después de tu scraping
df_libros = pd.DataFrame(data)

# 🧹 LIMPIEZA DE DATOS
# Limpiar precios: '£19.99' → 19.99
df_libros['precio_limpio'] = (
    df_libros['Precio']
    .str.replace(r'[^\d\.]', '', regex=True)
    .astype(float))

# Convertir ratings: 'Three' → 3
rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
df_libros['rating_numerico'] = df_libros['Rating'].map(rating_map).fillna(0)

# Limpiar stock: 'In stock (22 available)' → 22
df_libros['stock_numerico'] = df_libros['Stock'].str.extract(r'(\d+)').astype(int).fillna(0)

# Ver qué tan sucios están tus datos
print("📊 Resumen de datos limpios:")
print(df_libros[['precio_limpio', 'rating_numerico', 'stock_numerico']].describe())

# Convertir de vuelta a lista de diccionarios para MySQL
libros_data_limpios = df_libros.to_dict('records')

# Breves analisis
# 🔍 EXPLORACIÓN RÁPIDA
print("📈 Categorías más populares:")
print(df_libros['Categoría'].value_counts().head())

print("\n💰 Distribución de precios:")
print(df_libros['precio_limpio'].describe())

print("\n⭐ Distribución de ratings:")
print(df_libros['rating_numerico'].value_counts().sort_index())

# Detectar problemas
print("\n🚨 Libros sin precio:")
print(df_libros[df_libros['precio_limpio'].isna()].shape[0])


# Contar libros con múltiples autores
df_libros['cantidad_autores'] = df_libros['Autores'].apply(len)

# Filtrar libros con más de 1 autor
libros_multiples_autores = df_libros[df_libros['cantidad_autores'] > 1]

print(f"\n📚 Libros con más de 1 autor: {len(libros_multiples_autores)}")
print("\nDistribución de cantidad de autores por libro:")
print(df_libros['cantidad_autores'].value_counts().sort_index())

# Mostrar algunos ejemplos
print("\nEjemplos de libros con múltiples autores:")
print(libros_multiples_autores[['Título', 'Autores']].head())

# DataFrame solo con columnas limpias y necesarias
df_para_sql = df_libros[[
    'Título',
    'Autores',
    'precio_limpio',  # Usar el precio limpio (no el original con '£')
    'Disponibilidad',
    'rating_numerico', # Usar el rating numérico (no el textual 'Three')
    'stock_numerico',  # Usar el stock como número (no el texto completo)
    'Categoría',
    'URL'
]].copy()

df_para_sql = df_para_sql.rename(columns={
    'Título': 'titulo',
    'precio_limpio': 'precio',
    'rating_numerico': 'rating',
    'stock_numerico': 'stock',
    'URL': 'url'
})


📊 Resumen de datos limpios:
       precio_limpio  rating_numerico  stock_numerico
count     1000.00000      1000.000000     1000.000000
mean        35.07035         2.923000        8.585000
std         14.44669         1.434967        5.654622
min         10.00000         1.000000        1.000000
25%         22.10750         2.000000        3.000000
50%         35.98000         3.000000        7.000000
75%         47.45750         4.000000       14.000000
max         59.99000         5.000000       22.000000
📈 Categorías más populares:
Categoría
Default           152
Nonfiction        110
Sequential Art     75
Add a comment      67
Fiction            65
Name: count, dtype: int64

💰 Distribución de precios:
count    1000.00000
mean       35.07035
std        14.44669
min        10.00000
25%        22.10750
50%        35.98000
75%        47.45750
max        59.99000
Name: precio_limpio, dtype: float64

⭐ Distribución de ratings:
rating_numerico
1    226
2    196
3    203
4    179
5    196

## Modo de hacer con MYSLQ (mysql-connector-python)

In [6]:
import mysql.connector
from mysql.connector import Error

def conectar_mysql():
    """Crear conexión a MySQL"""
    try:
        conexion = mysql.connector.connect(
            host='localhost',       
            user='root',            # Usuario por defecto en tu contenedor
            password='12345',       # Contraseña
            database='libreria_books' # Base creada en Docker
        )
        
        if conexion.is_connected():
            print("✅ Conexión exitosa a MySQL (Docker)")
            return conexion
            
    except Error as e:
        print(f"❌ Error conectando a MySQL: {e}")
        return None


def crear_tablas_mysql(conexion):
    """Crear tablas en MySQL con estructura mejorada"""
    cursor = conexion.cursor()
    
    try:
        conexion.start_transaction()
        
        # Tabla categorias
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS categorias (
                id INT AUTO_INCREMENT PRIMARY KEY,
                nombre VARCHAR(100) NOT NULL UNIQUE,
                descripcion TEXT,
                creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                actualizado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP 
                    ON UPDATE CURRENT_TIMESTAMP
            ) ENGINE=InnoDB
        """)
        
        # Tabla libros
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS libros (
                id INT AUTO_INCREMENT PRIMARY KEY,
                titulo VARCHAR(255) NOT NULL,
                precio DECIMAL(10,2) NOT NULL,
                rating TINYINT CHECK (rating BETWEEN 0 AND 5),
                stock INT DEFAULT 0 NOT NULL,
                url VARCHAR(512),
                categoria_id INT NOT NULL,
                creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                actualizado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP 
                    ON UPDATE CURRENT_TIMESTAMP,
                FOREIGN KEY (categoria_id) REFERENCES categorias(id)
                    ON DELETE RESTRICT
            ) ENGINE=InnoDB
        """)
        
        # Tabla autores
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS autores (
                id INT AUTO_INCREMENT PRIMARY KEY,
                nombre VARCHAR(255) NOT NULL,
                biografia TEXT,
                nacionalidad VARCHAR(100),
                creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                actualizado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP 
                    ON UPDATE CURRENT_TIMESTAMP
            ) ENGINE=InnoDB
        """)
        
        # Tabla relación muchos a muchos
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS libro_autor (
                libro_id INT NOT NULL,
                autor_id INT NOT NULL,
                PRIMARY KEY (libro_id, autor_id),
                FOREIGN KEY (libro_id) REFERENCES libros(id)
                    ON DELETE CASCADE,
                FOREIGN KEY (autor_id) REFERENCES autores(id)
                    ON DELETE CASCADE
            ) ENGINE=InnoDB
        """)
        
        # Crear índices
        # cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_titulo ON libros(titulo)")
        # cursor.execute("CREATE INDEX IF NOT EXISTS idx_autores_nombre ON autores(nombre)")
        
        conexion.commit()
        print("✅ Tablas creadas exitosamente y relacion creada")
        
    except Exception as e:
        conexion.rollback()
        raise Exception(f"Error al crear tablas: {str(e)}")
    finally:
        cursor.close()
    
    

def insertar_datos_mysql(conexion, df_para_sql):
    """Versión mejorada para insertar todos los libros"""
    cursor = conexion.cursor()
    
    try:
        # Convertir DataFrame y asegurar nombres de columnas
        libros_data = df_para_sql.rename(columns={
            'Categoría': 'categoria',
            'Autores': 'autores'
        }).to_dict('records') if isinstance(df_para_sql, pd.DataFrame) else df_para_sql
        
        # Estadísticas
        total_libros = len(libros_data)
        libros_insertados = 0
        relaciones_creadas = 0
        
        # 1. Procesar todas las categorías primero
        categorias_unicas = {libro.get('categoria', 'Sin categoría').strip() 
                            for libro in libros_data if libro.get('categoria')}
        
        for categoria in categorias_unicas:
            cursor.execute("INSERT IGNORE INTO categorias (nombre) VALUES (%s)", (categoria,))
        conexion.commit()
        
        # 2. Insertar libros y autores
        for libro in libros_data:
            try:
                # Obtener categoría (con valor por defecto)
                categoria_nombre = libro.get('categoria', 'Sin categoría').strip()
                
                # Obtener ID de categoría (insensible a mayúsculas)
                cursor.execute("SELECT id FROM categorias WHERE LOWER(nombre) = LOWER(%s)", 
                            (categoria_nombre,))
                categoria_id = cursor.fetchone()[0]
                
                # Insertar libro
                cursor.execute("""
                    INSERT INTO libros (titulo, precio, rating, stock, url, categoria_id)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    libro.get('titulo', 'Sin título').strip(),
                    float(libro.get('precio', 0)),
                    int(libro.get('rating', 0)),
                    int(libro.get('stock', 0)),
                    libro.get('url', ''),
                    categoria_id
                ))
                libro_id = cursor.lastrowid
                libros_insertados += 1
                
                # Procesar autores (con manejo de varios formatos)
                autores = libro.get('autores', [])
                if isinstance(autores, str):  # Por si viene como string
                    autores = [a.strip() for a in autores.split(',') if a.strip()]
                
                for autor_nombre in autores:
                    if autor_nombre and isinstance(autor_nombre, str):
                        cursor.execute("INSERT IGNORE INTO autores (nombre) VALUES (%s)", 
                                    (autor_nombre.strip(),))
                        
                        cursor.execute("SELECT id FROM autores WHERE nombre = %s", 
                                    (autor_nombre.strip(),))
                        autor_id = cursor.fetchone()[0]
                        
                        cursor.execute("""
                            INSERT IGNORE INTO libro_autor (libro_id, autor_id)
                            VALUES (%s, %s)
                        """, (libro_id, autor_id))
                        relaciones_creadas += 1
                        
            except Exception as e:
                conexion.rollback()  # Rollback de la transacción del libro actual
                continue  # Continuar con el siguiente libro
        
        conexion.commit()
        
        # Reporte final
        print(f"\n📊 RESUMEN DE INSERCIÓN:")
        print(f"Total de libros procesados: {total_libros}")
        print(f"Libros insertados correctamente: {libros_insertados}")
        print(f"Relaciones libro-autor creadas: {relaciones_creadas}")
        print(f"Porcentaje de éxito: {(libros_insertados/total_libros)*100:.2f}%")
        
    finally:
        cursor.close()

# Ejemplo de uso
conexion = conectar_mysql()
if conexion:
    crear_tablas_mysql(conexion)
    insertar_datos_mysql(conexion, df_para_sql)
    conexion.close()

✅ Conexión exitosa a MySQL (Docker)
✅ Tablas creadas exitosamente y relacion creada

📊 RESUMEN DE INSERCIÓN:
Total de libros procesados: 1000
Libros insertados correctamente: 1000
Relaciones libro-autor creadas: 1115
Porcentaje de éxito: 100.00%


## Modo de hacer con SQL Alchemy 


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

class Categoria(Base):
    __tablename__ = 'categorias'
    
    id = Column(Integer, primary_key=True)
    nombre = Column(String(100), unique=True, nullable=False)
    descripcion = Column(Text)
    
    # Relación
    libros = relationship("Libro", back_populates="categoria")

class Autor(Base):
    __tablename__ = 'autores'
    
    id = Column(Integer, primary_key=True)
    nombre = Column(String(255), nullable=False)
    biografia = Column(Text)

class Libro(Base):
    __tablename__ = 'libros'
    
    id = Column(Integer, primary_key=True)
    titulo = Column(String(500), nullable=False)
    precio = Column(Float)
    rating = Column(Integer)  # 1-5
    stock = Column(Integer, default=0)
    url = Column(Text)
    categoria_id = Column(Integer, ForeignKey('categorias.id'))
    
    # Relaciones
    categoria = relationship("Categoria", back_populates="libros")
    autores = relationship("Autor", secondary="libro_autor", back_populates="libros")

# Tabla de relación muchos a muchos
from sqlalchemy import Table
libro_autor = Table('libro_autor', Base.metadata,
    Column('libro_id', Integer, ForeignKey('libros.id')),
    Column('autor_id', Integer, ForeignKey('autores.id'))
)

# Configurar la relación en Autor
Autor.libros = relationship("Libro", secondary=libro_autor, back_populates="autores")

### Consultas mas lindas con (ipython-sql)

In [ ]:
%load_ext sql



### Graficos Opcional con (matplotlib)